<a href="https://colab.research.google.com/github/Ryan-alb/MachineLearning/blob/main/notebooks/assitenteVirtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **O CÓDIGO A SEGUIR NÃO FUNCIONA SE EXECUTADO NO COLAB, POR ISSO RODE LOCALMENTE**

In [ ]:
!pip install SpeechRecognition pyttsx3 wikipedia pywhatkit sounddevice requests

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import speech_recognition as sr
import pyttsx3
import webbrowser
import wikipedia
import requests
import os

# Configurar o TTS
engine = pyttsx3.init()
engine.setProperty('rate', 150)

def speak(text):
    print(f"Assistente: {text}")
    engine.say(text)
    engine.runAndWait()

def gravar_audio(filename="output.wav", duration=5, fs=44100):
    speak("Estou ouvindo, pode falar agora.")
    print("Gravando...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    write(filename, fs, audio)
    print("Gravação finalizada.")

def reconhecer_audio(filename="output.wav"):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio_data = recognizer.record(source)
    try:
        texto = recognizer.recognize_google(audio_data, language='pt-BR')
        print(f"Você disse: {texto}")
        return texto.lower()
    except sr.UnknownValueError:
        speak("Não consegui entender.")
        return ""
    except sr.RequestError:
        speak("Problema ao acessar o serviço de reconhecimento.")
        return ""

def get_weather(city="São Paulo"):
    api_key = "SUA_API_KEY_AQUI"  # Pegue em openweathermap.org
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&lang=pt&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temp = data["main"]["temp"]
        description = data["weather"][0]["description"]
        return f"Agora em {city}, está {description} com temperatura de {temp} graus."
    else:
        return "Não consegui obter a previsão do tempo."

def assistant():
    speak("Olá! Estou pronta para te ajudar.")
    while True:
        gravar_audio()
        command = reconhecer_audio()

        if "wikipedia" in command:
            topic = command.replace("wikipedia", "").strip()
            try:
                result = wikipedia.summary(topic, sentences=2, lang='pt')
                speak(result)
            except:
                speak("Não encontrei nada sobre isso na Wikipedia.")

        elif "youtube" in command:
            speak("Abrindo o YouTube.")
            webbrowser.open("https://www.youtube.com")

        elif "farmácia" in command:
            speak("Procurando a farmácia mais próxima.")
            webbrowser.open("https://www.google.com/maps/search/farmácia+mais+próxima/")

        elif "amazon" in command:
            produtos = ["mochila", "relógio", "celular", "cadeira"]
            for produto in produtos:
                if produto in command:
                    speak(f"Procurando {produto} na Amazon.")
                    webbrowser.open(f"https://www.amazon.com.br/s?k={produto}")
                    break
            else:
                speak("Desculpe, só posso pesquisar mochila, relógio, celular ou cadeira.")

        elif "clima" in command:
            clima = get_weather()
            speak(clima)

        elif "sair" in command or "encerrar" in command:
            speak("Encerrando, até logo!")
            break

        else:
            speak("Comando não reconhecido.")

        # Limpar o arquivo de áudio temporário
        if os.path.exists("output.wav"):
            os.remove("output.wav")

# Rodar a assistente
assistant()
